In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly as py
import plotly.graph_objects as go
import plotly.express as px
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
py.offline.init_notebook_mode(connected = False)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import matplotlib.pyplot as plt
import seaborn as sns
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current sessionn

In [ ]:
ecom = pd.read_csv("../input/pakistans-largest-ecommerce-dataset/Pakistan Largest Ecommerce Dataset.csv",
                   parse_dates=["created_at", "Working Date"]
                   , low_memory=False)

In [ ]:
df = ecom.iloc[:,: -5]
df.isna().sum()
df.dropna(how="all",inplace=True)
df.columns = df.columns.str.lower()
df.columns = df.columns.str.strip().str.replace(" ","_")
df["mv"] = df["mv"].str.replace(",", "").str.replace("-","000").astype(float)

In [ ]:
df["month_name"] = df["created_at"].dt.month_name()
df["week_day_name"] = df["created_at"].dt.day_name()
df["week_day"] = df["created_at"].dt.weekday
df["week"] = df["created_at"].dt.isocalendar().week
df["month_start"] = df["created_at"].dt.is_month_start
df["month_end"]= df["created_at"].dt.is_month_end
df["quarter"] = df["created_at"].dt.quarter
df["quarter_start"] = df["created_at"].dt.is_quarter_end
df["quarter_end"]= df["created_at"].dt.is_quarter_start

df["year_start"] = df["created_at"].dt.is_year_start
df["year_end"] = df["created_at"].dt.is_year_end
df["month"] = df["created_at"].dt.month

# Extracting Day wise order count

In [ ]:
dto = df.groupby("created_at").size().reset_index(name="y")
dto = dto.rename(columns={"created_at":"ds","y":"y"})

<div style="background-color:#edd4c5;" /div>
<h1 style="font-family:verdana;"> <center> Using Facebook Prophet to forecast number of orders </center> </h1>
<h3><center style="color:#159364; font-family:cursive;"> Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.</center></h3>

In [ ]:
model = Prophet()
model.fit(dto)

In [ ]:
future = model.make_future_dataframe(periods=180) # predicting out of sample for next 180 days
future.tail(4)

# Lets Predict

In [ ]:
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
fig1 = model.plot(forecast,xlabel="Date",ylabel="Number of Orders")

# Incorporating Holidays effect

In [ ]:
holidays = pd.DataFrame(['2016-08-14', '2017-08-14', '2018-08-14']) # defining holiday for indepdence day 
holidays["holiday"] = 'Independence day'
holidays = holidays.rename(columns={0:"ds"})

In [ ]:
model_2 = Prophet(holidays=holidays)
model_2.add_country_holidays(country_name='PK') # FB prophet builtin holiday method to add hoildays specifically for paksitan
model_2.fit(dto)
forecast_holiday = model_2.predict(future)
fig2 = model_2.plot_components(forecast_holiday)
fig3 = model_2.plot(forecast_holiday)
a = add_changepoints_to_plot(fig3.gca(), model_2, forecast_holiday) # incorporating trend change_points 

# Evaluating Performance

In [ ]:
from fbprophet.diagnostics import cross_validation, performance_metrics #details: https://facebook.github.io/prophet/docs/diagnostics.html
from fbprophet.plot import add_changepoints_to_plot, plot_cross_validation_metric
p = Prophet()
p.fit(dto)

# Execute cross validation
df_cv = cross_validation(p, initial='365 days', period='180 days', horizon = '120 days')
pm = performance_metrics(df_cv, rolling_window=0.1)
display(pm.head(),pm.tail())
fig = plot_cross_validation_metric(df_cv, metric='mape', rolling_window=0.1)
plt.show()


<!-- <div class="alert alert-block alert-info" style="font-size:14px; font-family:verdana; line-height: 1.7em;">
    📌 &nbsp; Please keep in mind that, I am a beginner and on my learning journey as most of you. I just want to share something that will be useful for many kagglers working hard on their notebooks. If you find this notebook useful in anyway, please upvote it so that it can reach a bigger audience. You can share it with your fellow kagglers.
</div> -->
    
<!--     space -->
<br><br> 
<blockquote><p style="font-size:16px; color:#159364; font-family:verdana;"> <code>To be Continue</code>, <code> As Forecast perforemce is not much satisfactory</code><code></code>😃</p></blockquote>